## Data Source
**MEV-Blocks API**

### Extraction Details
- **Flashbots Block Data:** Includes miner reward and exogenous variables such as gas price, gas used, coinbase transfers, etc.
- **Flashbots Transaction Data:** Focuses on transactions within Flashbots bundles.

In [2]:
import requests as r
import pandas as pd
import numpy as np

In [4]:
def fetch_blocks(limit, before=None):
    url = f'https://blocks.flashbots.net/v1/blocks?limit={limit}'
    if before:
        url += f'&before={before}'
    response = r.get(url)
    return response.json()

def process_blocks(blocks_data):
    block_rows = []
    transaction_rows = []

    for block in blocks_data['blocks']:
        transactions = block.pop('transactions', [])  # Remove and retrieve transactions
        block_rows.append(block)

        for transaction in transactions:
            transaction['block_number'] = block['block_number']  # Add block_number to each transaction
            transaction_rows.append(transaction)

    return block_rows, transaction_rows


In [8]:
def main():
    total_blocks_needed = 10000
    limit_per_request = 100
    earliest_block = None
    all_block_rows = []
    all_transaction_rows = []

    while len(all_block_rows) < total_blocks_needed:
        blocks_data = fetch_blocks(limit_per_request, earliest_block)
        block_rows, transaction_rows = process_blocks(blocks_data)
        
        all_block_rows.extend(block_rows)
        all_transaction_rows.extend(transaction_rows)

        if block_rows:
            temp_df = pd.DataFrame(block_rows)
            earliest_block = temp_df['block_number'].iloc[-1]

    blocks_df = pd.DataFrame(all_block_rows)
    transactions_df = pd.DataFrame(all_transaction_rows)

    # Display the first few rows of each dataframe
    display(blocks_df.head())
    display(transactions_df.head())

    return blocks_df, transactions_df

In [9]:
blocks_df, transactions_df = main()

blocks_df.to_csv('blocks_data_api.csv', index=False)
transactions_df.to_csv('transactions_data_api.csv', index=False)

,block_number,miner_reward,fee_recipient_eth_diff,miner,fee_recipient,coinbase_transfers,eth_sent_to_fee_recipient,gas_used,gas_price,effective_priority_fee
0,18909018,23080669700625633,23080669700625633,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,9660228,2389246889,2389246889
1,18908989,27351715076158836,27351715076158836,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,28498662,959754358,959754358
2,18908901,79820385555004577,79820385555004577,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,13562433,5885403124,5885403124
3,18908883,31976058919197407,31976058919197407,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,1233105484329853,1233105484329853,8524681,3750997711,3750997711
4,18908715,116366731185440135,116366731185440135,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5,0,0,13903506,8369596214,8369596214


,transaction_hash,tx_index,bundle_type,bundle_index,block_number,eoa_address,to_address,gas_used,gas_price,coinbase_transfer,eth_sent_to_fee_recipient,total_miner_reward,fee_recipient_eth_diff
0,0x3086eb1f08ca12482c6fa1077834c3332fd753d13be8...,0,mempool,0,18909018,0x9FF8Ffbd465801161Fd45f56ADC2C113ed5EfF0f,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,65625,63531871790,0,0,4169279086218750,4169279086218750
1,0xd6c971f436f3ff1c7573ac0980fc7e7c4ba7627343de...,1,mempool,0,18909018,0x8C44EEe0d0081Df5F863Bb57889F222251FbAB63,0x0016C0d0343e8f2c3A7b6A51606B84B1545Ec606,21000,58531871790,0,0,1229169307590000,1229169307590000
2,0xb567d24a78a09e44b3182ae3a8895206883048404d55...,2,mempool,0,18909018,0x95295e831d11F131a582bec4aBb500C81D124Cdd,0xd784304969aeF13cB6D59B4C2661BC9692C394D4,21000,33531871790,0,0,704169307590000,704169307590000
3,0x21d09ffb0c10189d68acb268cf21f9579f758b71a443...,3,mempool,0,18909018,0x1B02039f86323a42D68fe7C00919dd1666481612,0xF3dE3C0d654FDa23daD170f0f320a92172509127,143743,16000000000,0,0,2299888000000000,2299888000000000
4,0xde7bdfeb625308b5efdab88f37eb133b6b18b6b5ca2b...,0,flashbots,1,18909018,0x4A3D63382d17361dBA7628743220285F9c169CE4,0x3328F7f4A1D1C57c35df56bBf0c9dCAFCA309C49,145421,15000000000,0,0,2181315000000000,2181315000000000
